# Envoy Workflows

This notebook is setup to help test the Envoy node using procedural steps representing each step.

## Setup

Make sure that you have PyEnvoy installed:

```
$ pip install pyenvoy
```

Create a credentials.json file with the URL, ClientID and ClientSecret for both of the Envoy nodes you'll be working with (e.g. envoy.local:8000 and counterparty.local:9000 or your TestNet endpoint and the Charlie rVASP). That file should look like:

```json
{
    "originator": {
        "url": "",
        "client_id": "",
        "client_secret": ""
    },
    "beneficiary": {
        "url": "",
        "client_id": "",
        "client_secret": ""
    }
}
```

## Credentials Loading and Connection

In [36]:
import json
import base64
import random

from envoy import connect

In [2]:
# Change this to the path of the credentials.json you created during setup!
CREDENTIALS_PATH = "./pyenvoy/.secret/credentials.json"


def load_credentials(name, path=CREDENTIALS_PATH):
    with open(path, 'r') as f:
        creds = json.load(f)
        return creds.get(name)


def connect_to(name, path=CREDENTIALS_PATH):
    creds = load_credentials(name, path)
    return connect(**creds)

In [3]:
# Connect to the originator
originator = connect_to("envoy.local")
originator.status()

{'status': 'ok', 'uptime': '13h49m44.804211417s', 'version': '0.24.0-beta.28'}

In [4]:
# Connect to beneficiary
beneficiary = connect_to("counterparty.local")
beneficiary.status()

{'status': 'ok', 'uptime': '13h50m10.817179959s', 'version': '0.24.0-beta.28'}

## Creating a Transaction

In this step we'll use the prepare and send prepared API endpoints to have the Envoy node help us create valid IVMS 101 formatted data.

In [11]:
# Change these variables to the ULIDs for your node(s)
# You can use originator.accounts.list() to list the accounts and when you retrieve an account
# use account.crypto_addresses.list() to return all the associated wallet addresses
ORIGINATOR_ACCOUNT_ID = "01J02D5YP4VAPVPC7Z9KS1PA3B"
BENEFICIARY_ACCOUNT_ID = "01J02D46KC1E485G5M060X8NF3"
ORIGINATOR_WALLET_ID = "01J02D5YP6163RW22BHCXX9YKY"
BENEFICIARY_WALLET_ID = "01J02D46KE3W8WNJ5W820GPZAN"


def prepare():
    originatorAccount = originator.accounts.detail(ORIGINATOR_ACCOUNT_ID)
    beneficiaryAccount = beneficiary.accounts.detail(BENEFICIARY_ACCOUNT_ID)

    originatorWallet = originatorAccount.crypto_addresses.detail(ORIGINATOR_WALLET_ID)
    beneficiaryWallet = beneficiaryAccount.crypto_addresses.detail(BENEFICIARY_WALLET_ID)

    return {
        "travel_address": beneficiaryAccount["travel_address"],
        "originator": {
            "first_name": originatorAccount["first_name"],
            "last_name": originatorAccount["last_name"],
            "customer_id": originatorAccount["customer_id"],
            "crypto_address": originatorWallet["crypto_address"],
            "country": "US",
        },
        "beneficiary": {
            "first_name": beneficiaryAccount["first_name"],
            "last_name": beneficiaryAccount["last_name"],
            "crypto_address": beneficiaryWallet["crypto_address"],
            "country": "DE",
        },
        "transfer": {"amount": random.random(), "network": "BTC"},
    }


def pprint(d):
    """
    Helper function to pretty print JSON responses
    """
    print(json.dumps(d, indent=2))

### Step 1: Create Prepared Data

In [12]:
prepared = originator.transactions.prepare(prepare())
pprint(prepared.data)

{
  "travel_address": "ta4E6vubFqJmFMeEEKZzaLWG8MkXYjKCc2uwajN2FH3TFXZ61fWmJ73buTsJ5vc35yJDx8fGtwaGZzkjN6rCn8fM4xud1MWfYH9ctkw9rKbKt",
  "identity": {
    "originator": {
      "originatorPersons": [
        {
          "naturalPerson": {
            "name": {
              "nameIdentifier": [
                {
                  "primaryIdentifier": "Tripp",
                  "secondaryIdentifier": "Michael",
                  "nameIdentifierType": "LEGL"
                }
              ]
            },
            "geographicAddress": [
              {
                "addressType": "HOME",
                "addressLine": [
                  "",
                  "",
                  ", ,"
                ],
                "country": "US"
              }
            ],
            "nationalIdentification": {},
            "customerIdentification": "62120997",
            "dateAndPlaceOfBirth": {},
            "countryOfResidence": "US"
          }
        }
      ],
      "accountNum

### Step 2: Send Prepared Data to Create a Transaction

In [14]:
txn = originator.transactions.send_prepared(prepared.data)
pprint(txn.data)

{
  "id": "8313dce3-223b-49f6-af6a-c8a881bb5dd8",
  "source": "local",
  "status": "pending",
  "counterparty": "Localhost Counterparty",
  "counterparty_id": "01J02CN8HD8KJK3H919M0N7ASG",
  "originator": "Michael Tripp",
  "originator_address": "n4mtuB4jtyNnPqznDcUfVMq4F8V3qRmVg3",
  "beneficiary": "Monika Kirsch",
  "beneficiary_address": "moL3rxQKoZj8zh8SmuKcCLHf2ifH15LhrP",
  "virtual_asset": "BTC",
  "amount": 0.7718436011065057,
  "last_update": "2024-08-16T14:21:22.834404Z",
  "created": "2024-08-16T09:21:22.804186-05:00",
  "modified": "2024-08-16T09:21:22.835153-05:00"
}


## Review the Incoming Transaction on the Beneficiary VASP 

In [16]:
# Use the transaction ID from the above txn to fetch it from the benficiary side instead.
txn = beneficiary.transactions.detail(txn["id"])
pprint(txn.data)

{
  "id": "8313dce3-223b-49f6-af6a-c8a881bb5dd8",
  "source": "remote",
  "status": "review",
  "counterparty": "Localhost Development",
  "counterparty_id": "01J02CNA91ENKD6TPFTBH33RVJ",
  "originator": "Michael Tripp",
  "originator_address": "n4mtuB4jtyNnPqznDcUfVMq4F8V3qRmVg3",
  "beneficiary": "Monika Kirsch",
  "beneficiary_address": "moL3rxQKoZj8zh8SmuKcCLHf2ifH15LhrP",
  "virtual_asset": "BTC",
  "amount": 0.7718436011065057,
  "last_update": "2024-08-16T14:21:22.820462Z",
  "envelope_count": 2,
  "created": "2024-08-16T09:21:22.807639-05:00",
  "modified": "2024-08-16T09:21:22.821307-05:00"
}


In [18]:
review = txn.accept_preview()
pprint(review.data)

{
  "id": "01J5DS10G5TEM2RQRQ0ZG9S4VJ",
  "envelope_id": "8313dce3-223b-49f6-af6a-c8a881bb5dd8",
  "direction": "out",
  "remote": "envoy.local",
  "reply_to": "01J5DS10G4KF0J5M8SN2MAC9R1",
  "is_error": false,
  "identity": {
    "originator": {
      "originatorPersons": [
        {
          "naturalPerson": {
            "name": {
              "nameIdentifier": [
                {
                  "primaryIdentifier": "Tripp",
                  "secondaryIdentifier": "Michael",
                  "nameIdentifierType": "LEGL"
                }
              ]
            },
            "geographicAddress": [
              {
                "addressType": "HOME",
                "addressLine": [
                  "",
                  "",
                  ", ,"
                ],
                "country": "US"
              }
            ],
            "nationalIdentification": {},
            "customerIdentification": "62120997",
            "dateAndPlaceOfBirth": {},
           

### Repair Workflow

At this point you could accept this transaction, but to show a full workflow, we're going to send a repair back to the originator to request changes.

In [20]:
# Note: "repair" --> "request_retry" depending on version of Envoy node
rep = txn.reject({"code": "MISSING_FIELDS", "message": "please supply date and place of birth", "repair": True})
pprint(rep.data)

{
  "id": "01J5DS9PPWNBKJG3N5C4A29S5Q",
  "envelope_id": "8313dce3-223b-49f6-af6a-c8a881bb5dd8",
  "direction": "in",
  "remote": "envoy.local",
  "reply_to": "01J5DS9PPV93CV7Z69AHDZHH3M",
  "is_error": true,
  "error": {
    "code": 153,
    "message": "please supply date and place of birth",
    "retry": true
  },
  "sent_at": null,
  "timestamp": "2024-08-16T14:26:07.698946Z",
  "transfer_state": "PENDING",
  "original": "CiQ4MzEzZGNlMy0yMjNiLTQ5ZjYtYWY2YS1jOGE4ODFiYjVkZDhKLAiZARIlcGxlYXNlIHN1cHBseSBkYXRlIGFuZCBwbGFjZSBvZiBiaXJ0aBgBUhsyMDI0LTA4LTE2VDE0OjI2OjA3LjY5ODk0NlpoAg=="
}


## Repair the Payload 

In [21]:
# Fetch the transaction from the originator side again 
txn = originator.transactions.detail(txn["id"])
pprint(txn.data)

{
  "id": "8313dce3-223b-49f6-af6a-c8a881bb5dd8",
  "source": "local",
  "status": "repair",
  "counterparty": "Localhost Counterparty",
  "counterparty_id": "01J02CN8HD8KJK3H919M0N7ASG",
  "originator": "Michael Tripp",
  "originator_address": "n4mtuB4jtyNnPqznDcUfVMq4F8V3qRmVg3",
  "beneficiary": "Monika Kirsch",
  "beneficiary_address": "moL3rxQKoZj8zh8SmuKcCLHf2ifH15LhrP",
  "virtual_asset": "BTC",
  "amount": 0.7718436011065057,
  "last_update": "2024-08-16T14:26:07.698946Z",
  "envelope_count": 6,
  "created": "2024-08-16T09:21:22.804186-05:00",
  "modified": "2024-08-16T09:26:07.699361-05:00"
}


In [22]:
# Get the repair preview to make changes 
repair = txn.repair_preview()
pprint(repair.data)

{
  "error": {
    "code": "MISSING_FIELDS",
    "message": "please supply date and place of birth",
    "repair": true
  },
  "envelope": {
    "id": "01J5DS10GJ9S6MCBYB7Z51BNSR",
    "envelope_id": "8313dce3-223b-49f6-af6a-c8a881bb5dd8",
    "direction": "in",
    "remote": "counterparty.local",
    "reply_to": "01J5DS10GJ9S6MCBYB7XEK8S4J",
    "is_error": false,
    "identity": {
      "originator": {
        "originatorPersons": [
          {
            "naturalPerson": {
              "name": {
                "nameIdentifier": [
                  {
                    "primaryIdentifier": "Tripp",
                    "secondaryIdentifier": "Michael",
                    "nameIdentifierType": "LEGL"
                  }
                ]
              },
              "geographicAddress": [
                {
                  "addressType": "HOME",
                  "addressLine": [
                    "",
                    "",
                    ", ,"
                  ],
    

In [28]:
# Create a response to make the repair
repaired = {
    "envelope_id": repair["envelope"]["envelope_id"],
    "identity": repair["envelope"]["identity"].data,
    "transaction": repair["envelope"]["pending"]["transaction"].data,
    "sent_at": repair["envelope"]["sent_at"],
    "received_at": repair["envelope"]["received_at"],
}

repaired["identity"]["originator"]["originatorPersons"][0]["naturalPerson"]["dateAndPlaceOfBirth"] = {
    "dateOfBirth": "1972-11-21",
    "placeOfBirth": "Smallville, KA, USA",
}

pprint(repaired)

{
  "envelope_id": "8313dce3-223b-49f6-af6a-c8a881bb5dd8",
  "identity": {
    "originator": {
      "originatorPersons": [
        {
          "naturalPerson": {
            "name": {
              "nameIdentifier": [
                {
                  "primaryIdentifier": "Tripp",
                  "secondaryIdentifier": "Michael",
                  "nameIdentifierType": "LEGL"
                }
              ]
            },
            "geographicAddress": [
              {
                "addressType": "HOME",
                "addressLine": [
                  "",
                  "",
                  ", ,"
                ],
                "country": "US"
              }
            ],
            "nationalIdentification": {},
            "customerIdentification": "62120997",
            "dateAndPlaceOfBirth": {
              "dateOfBirth": "1972-11-21",
              "placeOfBirth": "Smallville, KA, USA"
            },
            "countryOfResidence": "US"
          }
    

In [29]:
rep = txn.repair(repaired)
pprint(rep.data)

{
  "id": "01J5DSM2VNW91KYGHPWJ9210DH",
  "envelope_id": "8313dce3-223b-49f6-af6a-c8a881bb5dd8",
  "direction": "in",
  "remote": "counterparty.local",
  "reply_to": "01J5DSM2VNW91KYGHPWFGDBXA9",
  "is_error": false,
  "identity": {
    "originator": {
      "originatorPersons": [
        {
          "naturalPerson": {
            "name": {
              "nameIdentifier": [
                {
                  "primaryIdentifier": "Tripp",
                  "secondaryIdentifier": "Michael",
                  "nameIdentifierType": "LEGL"
                }
              ]
            },
            "geographicAddress": [
              {
                "addressType": "HOME",
                "addressLine": [
                  "",
                  "",
                  ", ,"
                ],
                "country": "US"
              }
            ],
            "nationalIdentification": {},
            "customerIdentification": "62120997",
            "dateAndPlaceOfBirth": {
       

## Accept with Modifications

In [30]:
txn = beneficiary.transactions.detail(txn["id"])

In [31]:
preview = txn.accept_preview()
pprint(preview.data)

{
  "id": "01J5DSM2V5ZTXAPDPR3TJTNNJV",
  "envelope_id": "8313dce3-223b-49f6-af6a-c8a881bb5dd8",
  "direction": "out",
  "remote": "envoy.local",
  "reply_to": "01J5DSM2V5ZTXAPDPR3RDZKZV8",
  "is_error": false,
  "identity": {
    "originator": {
      "originatorPersons": [
        {
          "naturalPerson": {
            "name": {
              "nameIdentifier": [
                {
                  "primaryIdentifier": "Tripp",
                  "secondaryIdentifier": "Michael",
                  "nameIdentifierType": "LEGL"
                }
              ]
            },
            "geographicAddress": [
              {
                "addressType": "HOME",
                "addressLine": [
                  "",
                  "",
                  ", ,"
                ],
                "country": "US"
              }
            ],
            "nationalIdentification": {},
            "customerIdentification": "62120997",
            "dateAndPlaceOfBirth": {
             

In [33]:
accepted = {
    "envelope_id": preview["envelope_id"],
    "identity": preview["identity"].data,
    "transaction": preview["pending"]["transaction"].data,
    "sent_at": preview["sent_at"],
    "received_at": preview["received_at"],
}

accepted["identity"]["beneficiary"]["beneficiaryPersons"][0]["naturalPerson"]["dateAndPlaceOfBirth"] = {
    "dateOfBirth": "1993-02-06",
    "placeOfBirth": "Hamburg, DE",
}

rep = txn.accept(accepted)
pprint(rep.data)

{
  "id": "01J5DSW0M52CE4H5WE9AC965T6",
  "envelope_id": "8313dce3-223b-49f6-af6a-c8a881bb5dd8",
  "direction": "in",
  "remote": "envoy.local",
  "reply_to": "01J5DSW0M52CE4H5WE991W9E2Z",
  "is_error": false,
  "identity": {
    "originator": {
      "originatorPersons": [
        {
          "naturalPerson": {
            "name": {
              "nameIdentifier": [
                {
                  "primaryIdentifier": "Tripp",
                  "secondaryIdentifier": "Michael",
                  "nameIdentifierType": "LEGL"
                }
              ]
            },
            "geographicAddress": [
              {
                "addressType": "HOME",
                "addressLine": [
                  "",
                  "",
                  ", ,"
                ],
                "country": "US"
              }
            ],
            "nationalIdentification": {},
            "customerIdentification": "62120997",
            "dateAndPlaceOfBirth": {
              

## Complete the Transaction

Now that the travel rule PII exchange has been accepted, perform the on-chain transaction and send back the txnid for record locating on the chain.

In [35]:
txn = originator.transactions.detail(txn["id"])
pprint(txn.data)

{
  "id": "8313dce3-223b-49f6-af6a-c8a881bb5dd8",
  "source": "local",
  "status": "accepted",
  "counterparty": "Localhost Counterparty",
  "counterparty_id": "01J02CN8HD8KJK3H919M0N7ASG",
  "originator": "Michael Tripp",
  "originator_address": "n4mtuB4jtyNnPqznDcUfVMq4F8V3qRmVg3",
  "beneficiary": "Monika Kirsch",
  "beneficiary_address": "moL3rxQKoZj8zh8SmuKcCLHf2ifH15LhrP",
  "virtual_asset": "BTC",
  "amount": 0.7718436011065057,
  "last_update": "2024-08-16T14:36:07.666512Z",
  "envelope_count": 10,
  "created": "2024-08-16T09:21:22.804186-05:00",
  "modified": "2024-08-16T09:36:07.667441-05:00"
}


In [38]:
def generate_bitcoin_hash():
    h = bytes([random.randrange(0, 256) for _ in range(0, 64)])
    return base64.b64encode(h)

In [39]:
latest = txn.latest_payload()
pprint(latest.data)

{
  "id": "01J5DSW0KKWQ9JEQQHTQKV0CB4",
  "envelope_id": "8313dce3-223b-49f6-af6a-c8a881bb5dd8",
  "direction": "out",
  "remote": "counterparty.local",
  "reply_to": "01J5DSW0KJNGMHDZDTNVKN8N45",
  "is_error": false,
  "identity": {
    "originator": {
      "originatorPersons": [
        {
          "naturalPerson": {
            "name": {
              "nameIdentifier": [
                {
                  "primaryIdentifier": "Tripp",
                  "secondaryIdentifier": "Michael",
                  "nameIdentifierType": "LEGL"
                }
              ]
            },
            "geographicAddress": [
              {
                "addressType": "HOME",
                "addressLine": [
                  "",
                  "",
                  ", ,"
                ],
                "country": "US"
              }
            ],
            "nationalIdentification": {},
            "customerIdentification": "62120997",
            "dateAndPlaceOfBirth": {
      

In [42]:
complete = {
    "envelope_id": latest["envelope_id"],
    "identity": latest["identity"].data,
    "transaction": latest["transaction"].data,
    "sent_at": latest["sent_at"],
    "received_at": latest["received_at"],
    "transfer_state": "COMPLETED",
}

complete["transaction"]["txid"] = generate_bitcoin_hash().decode("utf-8")

rep = txn.send(complete)
pprint(rep.data)

{
  "id": "01J5DTA16PPPFACP9R3W7T5MTT",
  "envelope_id": "8313dce3-223b-49f6-af6a-c8a881bb5dd8",
  "direction": "in",
  "remote": "counterparty.local",
  "reply_to": "01J5DTA16PPPFACP9R3VXVZ7MH",
  "is_error": false,
  "identity": {
    "originator": {
      "originatorPersons": [
        {
          "naturalPerson": {
            "name": {
              "nameIdentifier": [
                {
                  "primaryIdentifier": "Tripp",
                  "secondaryIdentifier": "Michael",
                  "nameIdentifierType": "LEGL"
                }
              ]
            },
            "geographicAddress": [
              {
                "addressType": "HOME",
                "addressLine": [
                  "",
                  "",
                  ", ,"
                ],
                "country": "US"
              }
            ],
            "nationalIdentification": {},
            "customerIdentification": "62120997",
            "dateAndPlaceOfBirth": {
       